In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
embeddings = None

[nltk_data] Downloading package punkt to /home/jorge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jorge/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jorge/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Spliteo para obtener vectores de train y test

In [6]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values
#train,test = train_test_split(train_df,test_size=0.33,random_state = 17)
#train.reset_index(inplace=True,drop=True)
#test.reset_index(inplace=True,drop=True)
#y_train = train['target'].values
#y_test = test['target'].values

In [3]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [4]:
processed, X_processed = prepare_for_cnn(train_df)

Percentage of words covered in the embeddings = 0.6336399642263958


In [5]:
processed

,invalid_location_character_count,location_is_place,mean_encode,keyword_length,text_length,location_length,stop_word_count,punctuation_count,hashtag_count,mention_count,...,text_embedding_290,text_embedding_291,text_embedding_292,text_embedding_293,text_embedding_294,text_embedding_295,text_embedding_296,text_embedding_297,text_embedding_298,text_embedding_299
0,0,0,0.585906,0,69,0,6,1,1,0,...,-0.055542,0.115112,-0.114197,-0.073990,-0.028381,0.010828,-0.061638,-0.080215,-0.020348,0.119019
1,0,0,0.585906,0,38,0,0,1,0,0,...,-0.059473,-0.018774,-0.040625,0.167126,0.067529,0.010791,-0.139258,-0.042334,0.070020,0.054913
2,0,0,0.585906,0,133,0,11,3,0,0,...,-0.068675,-0.000366,-0.106979,-0.019196,0.020692,0.031937,-0.069041,0.020345,0.021281,-0.016398
3,0,0,0.585906,0,65,0,1,2,1,0,...,0.208557,-0.069417,-0.071248,0.039714,0.195394,0.079915,-0.236613,0.025553,-0.026367,-0.033366
4,0,0,0.585906,0,88,0,7,2,2,0,...,0.080957,-0.128174,-0.070508,0.061353,0.019550,-0.023505,0.016354,-0.053082,0.030786,0.031708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0.585906,0,83,0,2,5,0,0,...,-0.127472,0.016769,-0.119431,-0.029610,0.010361,-0.045746,-0.038895,-0.090240,-0.001657,-0.127274
7609,0,0,0.585906,0,125,0,9,5,0,2,...,0.004173,0.020516,-0.027425,0.006314,0.001776,-0.016901,-0.112602,-0.011907,0.046770,-0.023856
7610,0,0,0.585906,0,65,0,2,11,0,0,...,0.174235,0.122884,-0.046224,0.060221,-0.155924,-0.039388,-0.201172,0.010946,0.118978,0.344401
7611,0,0,0.585906,0,137,0,5,5,0,0,...,0.090700,0.071742,-0.048296,0.061262,-0.116019,-0.001737,-0.046749,-0.062116,0.018721,-0.048802


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [12]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)])
    return model

In [13]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 128)               202752    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 211,073
Trainable params: 211,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)

Epoch 1/50
128/128 [==============================] - 1s 9ms/step - loss: 0.3188 - accuracy: 0.8578 - val_loss: 0.5273 - val_accuracy: 0.7775
Epoch 2/50
128/128 [==============================] - 1s 12ms/step - loss: 0.3171 - accuracy: 0.8588 - val_loss: 0.5233 - val_accuracy: 0.7775
Epoch 3/50
128/128 [==============================] - 1s 9ms/step - loss: 0.3160 - accuracy: 0.8620 - val_loss: 0.5363 - val_accuracy: 0.7725
Epoch 4/50
105/128 [=======================>......] - ETA: 0s - loss: 0.3150 - accuracy: 0.8604

In [15]:
prediction = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [16]:
accuracy_score(y_test, prediction)

0.8042180660565061

In [17]:
f1_score(y_test, prediction)

0.7368983957219252